# Quality Evaluators with the Azure AI Evaluation SDK
The following sample shows the basic way to evaluate a Generative AI application in your development environment with the Azure AI evaluation SDK.

> ✨ ***Note*** <br>
> Please check the reference document before you get started - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk

## 🔨 Current Support and Limitations (as of 2025-01-14) 
- Check the region support for the Azure AI Evaluation SDK. https://learn.microsoft.com/en-us/azure/ai-studio/concepts/evaluation-metrics-built-in?tabs=warning#region-support

### Region support for evaluations
| Region              | Hate and Unfairness, Sexual, Violent, Self-Harm, XPIA, ECI (Text) | Groundedness (Text) | Protected Material (Text) | Hate and Unfairness, Sexual, Violent, Self-Harm, Protected Material (Image) |
|---------------------|------------------------------------------------------------------|---------------------|----------------------------|----------------------------------------------------------------------------|
| North Central US    | no                                                               | no                  | no                         | yes                                                                        |
| East US 2           | yes                                                              | yes                 | yes                        | yes                                                                        |
| Sweden Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| US North Central    | yes                                                              | no                  | yes                        | yes                                                                        |
| France Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| Switzerland West    | yes                                                              | no                  | no                         | yes                                                                        |

### Region support for adversarial simulation
| Region            | Adversarial Simulation (Text) | Adversarial Simulation (Image) |
|-------------------|-------------------------------|---------------------------------|
| UK South          | yes                           | no                              |
| East US 2         | yes                           | yes                             |
| Sweden Central    | yes                           | yes                             |
| US North Central  | yes                           | yes                             |
| France Central    | yes                           | no                              |


## ✔️ Pricing and billing
- Effective 1/14/2025, Azure AI Safety Evaluations will no longer be free in public preview. It will be billed based on consumption as following:

| Service Name              | Safety Evaluations       | Price Per 1K Tokens (USD) |
|---------------------------|--------------------------|---------------------------|
| Azure Machine Learning    | Input pricing for 3P     | $0.02                     |
| Azure Machine Learning    | Output pricing for 3P    | $0.06                     |
| Azure Machine Learning    | Input pricing for 1P     | $0.012                    |
| Azure Machine Learning    | Output pricing for 1P    | $0.012                    |


In [ ]:
import pandas as pd
import os
import json

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator
from azure.ai.evaluation import GroundednessEvaluator, GroundednessProEvaluator
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation,
    Dataset,
    EvaluatorConfiguration,
    ConnectionType,
    EvaluationSchedule,
    RecurrenceTrigger,
    ApplicationInsightsConfiguration
)
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
    F1ScoreEvaluator,
    RetrievalEvaluator
)

from azure.ai.ml import MLClient



load_dotenv()

In [ ]:
credential = DefaultAzureCredential()

azure_ai_project_conn_str = os.environ.get("AZURE_AI_PROJECT_CONN_STR")
subscription_id = azure_ai_project_conn_str.split(";")[1]
resource_group_name = azure_ai_project_conn_str.split(";")[2]
project_name = azure_ai_project_conn_str.split(";")[3]

azure_ai_project_dict = {
    "subscription_id": subscription_id,
    "resource_group_name": resource_group_name,
    "project_name": project_name,
}

azure_ai_project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=azure_ai_project_conn_str
)


model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "azure_openai",
}

## 🚀 Generate synthetic dataset with Azure OpenAI
- Use your models to generate custom textual utterances for your purpose in your target language. These utterances serve as a seed for the evaluation creation. By adjusting your prompts, you can produce text tailored to your domain (such as call center Q&A for a tech brand).

In [ ]:
from openai import AzureOpenAI


aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

try:
    client = AzureOpenAI(
        azure_endpoint = aoai_api_endpoint,
        api_key        = aoai_api_key,
        api_version    = aoai_api_version,
    )

    print("=== Initialized AzuureOpenAI client ===")
    print(f"AZURE_OPENAI_ENDPOINT={aoai_api_endpoint}")
    print(f"AZURE_OPENAI_API_VERSION={aoai_api_version}")
    print(f"AZURE_OPENAI_DEPLOYMENT_NAME={aoai_deployment_name}")
        
except (ValueError, TypeError) as e:
    print(e)

In [ ]:
NUM_SAMPLES = 20

LANGUAGE = "English"

topic = f"""
Contoso Electronics call center QnA related expected spoken utterances for {LANGUAGE} languages. create {NUM_SAMPLES} lines of jsonl of the topic related text.
"""

example = """
Here is examples of the expected format:
{"query":"How do you create a run?","context":"AML API only","response":"To create a run using the Azure Machine Learning API, you first need to create an Experiment. Once you have an experiment, you can create a Run object that is associated with that experiment. Here is some Python code that demonstrates this process:\\n\\n```\\nfrom azureml.core import Experiment, Run\\nfrom azureml.core.workspace import Workspace\\n\\n# Define workspace and experiment\\nws = Workspace.from_config()\\nexp = Experiment(workspace=ws, name='my_experiment')\\n\\n# Create a new run\\nrun = exp.start_logging()\\n```\\n\\nIn this code, the `from_config()` method reads the configuration file that you created when you set up your Azure Machine Learning workspace. The `Experiment` constructor creates an Experiment object that is associated with your workspace, and the `start_logging()` method creates a new Run object that is associated with the Experiment. Now you can use the `run` object to log metrics, upload files, and track other information related to your machine learning experiment.","ground_truth":"Paris is the capital of France."}
{"query":"How do you log a model?","context":"Logging can be done using any OSS Sdk","response":"There are a few ways to log models in Azure Machine Learning. \\n\\nOne way is to use the `register_model()` method of the `Run` object. The `register_model()` method logs a model file in the Azure Machine Learning service workspace and makes it available for deployment. Here's an example:\\n\\n```python\\nfrom azureml.core import Model\\n\\nmodel_path = './outputs/my_model.pkl'\\nmodel = Model.register(workspace=ws, model_path=model_path, model_name='my_model')\\n```\\n\\nThis code registers the model file located at `model_path` to the Azure Machine Learning service workspace with the name `my_model`. \\n\\nAnother way to log a model is to save it as an output of a `Run`. If your model generation code is part of a script or Jupyter notebook that runs as an Azure Machine Learning experiment, you can save the model file as an output of the `Run` object. Here's an example:\\n\\n```python\\nfrom sklearn.linear_model import LogisticRegression\\nfrom azureml.core.run import Run\\n\\n# Initialize a run object\\nrun = Run.get_context()\\n\\n# Train your model\\nX_train, y_train = ...\\nlog_reg = LogisticRegression().fit(X_train, y_train)\\n\\n# Save the model to the Run object's outputs directory\\nmodel_path = 'outputs/model.pkl'\\njoblib.dump(value=log_reg, filename=model_path)\\n\\n# Log the model as a run artifact\\nrun.upload_file(name=model_path, path_or_stream=model_path)\\n```\\n\\nIn this code, `Run.get_context()` retrieves the current run context object, which you can use to track metadata and metrics for the run. After training your model, you can use `joblib.dump()` to save the model to a file, and then log the file as an artifact of the run using `run.upload_file()`.","ground_truth":"Paris is the capital of France."}
{"query":"What is the capital of France?","context":"France is in Europe","response":"Paris is the capital of France.","ground_truth":"Paris is the capital of France."}
"""

system_message = """
Generate plain text sentences of #topic# related text to improve the recognition of domain-specific words and phrases.
Domain-specific words can be uncommon or made-up words, but their pronunciation must be straightforward to be recognized. 
Use text data that's close to the expected spoken utterances. The nummber of utterances per line should be 1. 
jsonl format is required. use 'no' as number, 'query' as string, 'context' as string, 'response' as string, and 'ground_truth' as string.
only include the lines as the result. Do not include ```jsonl, ``` and blank line in the result. 

"""

user_message = f"""
#topic#: {topic}
Example: {example}
"""

# Simple API Call
response = client.chat.completions.create(
    model=aoai_deployment_name,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.8,
    top_p=0.1
)

content = response.choices[0].message.content
print(content)
print("Usage Information:")
#print(f"Cached Tokens: {response.usage.prompt_tokens_details.cached_tokens}") #only o1 models support this
print(f"Completion Tokens: {response.usage.completion_tokens}")
print(f"Prompt Tokens: {response.usage.prompt_tokens}")
print(f"Total Tokens: {response.usage.total_tokens}")

In [ ]:
synthetic_text_file = "data/sythetic_evaluation_data.jsonl"
with open(synthetic_text_file, 'w', encoding='utf-8') as f:
    for line in content.split('\n'):
        if line.strip():  # Check if the line is not empty
            f.write(line + '\n')

%store synthetic_text_file

## 🚀 Run Evaluators in Azure Cloud (azure.ai.evaluation.evaluate)

In [ ]:
input_path = "data/evaluate_test_data.jsonl"
output_path = "data/cloud_evaluation_output.json"

# https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/flow-evaluate-sdk
retrieval_evaluator = RetrievalEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)

column_mapping = {
    "query": "${data.query}",
    "ground_truth": "${data.ground_truth}",
    "response": "${data.response}",
    "context": "${data.context}",
}

In [ ]:
import time
from datetime import datetime
result = evaluate(
    evaluation_name=f"evaluation_cloud_{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    data=synthetic_text_file,
    evaluators={
        "retrieval": retrieval_evaluator,
        "fluency": fluency_evaluator,
        "groundedness": groundedness_evaluator,
        "relevance": relevance_evaluator,
        "coherence": coherence_evaluator,
        "similarity": similarity_evaluator
    },
    evaluator_config={
        "retrieval": {"column_mapping": column_mapping},
        "relevance": {"column_mapping": column_mapping},
        "similarity": {"column_mapping": column_mapping},
        "groundedness": {"column_mapping": column_mapping},
        "coherence": {"column_mapping": column_mapping},
        "similarity": {"column_mapping": column_mapping},
    },
    azure_ai_project=azure_ai_project_dict,
    output_path=output_path,
)
